In [ ]:
# Install and import
import os, sys, numpy as np
from glob import glob
try: # Check platform (Colab or Jupyter)
  import google.colab
  my_drive = "/content/drive/My Drive/"
except:
  my_drive = os.path.expanduser("~") + "/drive/"
path = my_drive + "joklar/"
sys.path.append(path + "src")
from util.util import install_import
from util.image_util import get_border_indices, load_data
rasterio = install_import("rasterio")

In [ ]:
# Define data to use
datafolder = "joklar/data"
dataname = "lang"
datapath = path + "data/" + dataname + "/"

In [ ]:
# Define functions
def read_tile(datapath, n):
    #dataname = os.path.basename(datapath.rstrip('/'))
    imgfile = datapath + f"/img-{n:03d}.tif"
    maskfile = datapath + f"/mask-{n:03d}.tif"
    with rasterio.open(imgfile) as f:
        img_channels = f.meta["count"]
        img = np.array([f.read(b) for b in range(1, img_channels + 1)])
        x = f.meta["transform"][2]
        y = f.meta["transform"][5]
    with rasterio.open(maskfile) as f:
        mask_channels = f.meta["count"]
        masks = np.array([f.read(b) for b in range(1, mask_channels + 1)])
        mask = np.any(masks, axis=0)
    return img, mask, x, y

In [ ]:
img, mask, x, y = read_tile(datapath, 21)
img.dtype, mask.dtype

In [ ]:
def read_tiles(datapath):
    # Read a whole glacier
    nfiles = len(glob(f"{datapath}/img-*"))
    img = nfiles*[None]
    mask = np.zeros((nfiles, 256, 256), bool)
    x = np.zeros(nfiles)
    y = np.zeros(nfiles)
    for n in range(nfiles):
        img[n], mask[n], x[n], y[n] = read_tile(datapath, n)
        if n % 10 == 0:
            print(f"{n}, ", end="")
    col = ((x - x.min())/2560).astype(int)
    row = ((y - y.min())/2560).astype(int)
    I = np.lexsort((col, row))
    row = row[I]
    col = col[I]
    mask = mask[I]
    data = np.transpose(np.array(img)[I], (0, 2, 3, 1))
    return data, mask, row, col

In [ ]:
def print_info(border_indices, image, mask, row, col):
    # Print summary and tif-meta information
    w = (col[-1] + 1)*2.56
    h = (row[-1] + 1)*2.56
    pct_glacier = np.sum(mask)/np.size(mask)*100
    n_border = len(border_indices)
    pct_border = len(border_indices)/len(mask)*100
    print()
    print(f"Image shape: {image.shape}, data type: {image.dtype}")
    print(f"Region width {w} km, height {h} km")
    print(f"Pixels labeled glacier: {pct_glacier:.1f}%")
    print(f"Border indices: {n_border} ({pct_border:.1f}%)")
    print(f"Mask datatype: {mask.dtype}")
    print()

def save_data_and_report(outfile, border_indices, image, mask, row, col):
    np.savez_compressed(outfile, border_indices=border_indices,
                        image=image, mask=mask, row=row, col=col)
    size = os.path.getsize(outfile)
    print(size)
    compress_ratio = size/(image.nbytes + mask.nbytes + row.nbytes + col.nbytes)
    print(f"Wrote {size/2**20:.1f} Mb of data to {outfile}")
    print(f"  Compression ratio: {compress_ratio:.2f}")
    print()

In [ ]:
# Read data and report
image, mask, row, col = read_tiles(datapath)
border_indices = get_border_indices(image, mask)
print_info(border_indices, image, mask, row, col)

In [ ]:
# Save to compressed file and report
dataname = os.path.basename(datapath.rstrip("/"))
outfile = datapath + "data.npz"
save_data_and_report(outfile, border_indices, image, mask, row, col)

In [ ]:
%%time
# Test reading of the file
(image, mask, row, col) = load_data(outfile, "all")
print(len(mask))
image.shape, mask.shape, row.shape, col.shape